In [18]:
!python Feature_Extract.py aes128_table_ecb key_in

!python Feature_Extract.py AES_PPRM1 Kin AES_PPRM1
!python Feature_Extract.py AES_PPRM3 Kin AES_PPRM3
!python Feature_Extract.py AES_TBL Kin AES_TBL
!python Feature_Extract.py RSA Kin RSA
!python Feature_Extract.py SABER pol_64bit_in SABER
# !python Feature_Extract.py aes128_table_ecb key_in train

/Users/ruixiang/PycharmProjects/SCAR/.venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(
Loading cached VCD object from: ../data/aes128_table_ecb/aes128_table_ecb_vcd.pkl
Loading toggle counts from cache: ../data/aes128_table_ecb/aes128_table_ecb_toggle.txt
Successfully loaded toggle counts from cache.
[INFO] Features written to ../out/features.csv
[INFO] Edges written to ../out/edges.csv
/Users/ruixiang/PycharmProjects/SCAR/.venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(
Loading cached VCD object from: ../data/AES_PPRM1/AES_PPRM1_vcd.pkl
Loading toggle counts from cache: ../data/AES_PPRM1/AES_P

In [29]:
!python SCAR_GNN.py

/Users/ruixiang/PycharmProjects/SCAR/.venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(
num_features: 7, num_classes: 2
feature_names: ['Degree', 'Hamming distance', 'Paths', 'and', 'mux', 'or', 'xor']
Train label distribution:
label
0    66
1    66
Name: count, dtype: int64

Test label distribution:
label
0    15
1     8
Name: count, dtype: int64
Epoch 1/32
6/6 ━━━━━━━━━━━━━━━━━━━━ 3s 48ms/step - acc: 0.5415 - loss: 5.4873 - precision: 0.4977 - recall: 0.6360 - val_acc: 0.1429 - val_loss: 3.2786 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00
Epoch 2/32
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - acc: 0.4820 - loss: 7.0970 - precision: 0.4573 - recall: 0.5725 - val_acc: 0.1429 - val_loss: 4.7287 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00
Epoch 3/32
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/

In [30]:
!python test.py

/Users/ruixiang/PycharmProjects/SCAR/.venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(
Found 5 dataset(s): ['AES_PPRM1', 'AES_PPRM3', 'AES_TBL', 'RSA', 'SABER']
Testing AES_PPRM1: read ../test/AES_PPRM1_features.csv and ../test/AES_PPRM1_edges.csv
Tensor("GatherV2:0", shape=(1, 32), dtype=float32)
Tensor("gnn_model_1/GatherV2:0", shape=(1, 32), dtype=float32)
Tensor("gnn_model_1/GatherV2:0", shape=(1, 32), dtype=float32)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 317ms/step
Tensor("gnn_model_1/GatherV2:0", shape=(None, 32), dtype=float32)
Testing AES_PPRM3: read ../test/AES_PPRM3_features.csv and ../test/AES_PPRM3_edges.csv
Tensor("GatherV2:0", shape=(1, 32), dtype=float32)
Tensor("gnn_model_1/GatherV2:0", shape=(1, 32), dtype=float32)
Tensor("gnn_model_1/GatherV2:0", shape=(1, 32), dtype=float32)
1/1 ━━━